In [1]:
#- 패키지 임포트
import pandas as pd
from tqdm import tqdm
import re
from konlpy.tag import Okt
okt = Okt()
from collections import Counter # 형태소별 빈도 구할 때 사용
from IPython.display import clear_output
from IPython import display # 램효율을 늘리기 위해 아웃풋 display를 지워줄 것.
from ipywidgets import Output

out = Output()
display.display(out)

Output()

In [2]:
''' 파일 불러오기 '''
df = pd.read_csv("../dataset/2022/본문_전처리/나무문화_본문_11월_전처리.txt", encoding='UTF-8')
print(df.shape, df.columns)
corpus = df['split_str'].to_list() # 문장(X문서X)으로 가져와서 실행.
# ↑ 추후에 다른 분석 시도할 때 문장 별 형태소가 필요할 때도 있음. 문장별 형태소와 이를 문서로 합친 게 있으면 편하기 때문에 문장별로 형태소 분석 실행.
print(len(corpus))

(1084190, 2) Index(['split_str', 'org_idx'], dtype='object')
1084190


In [3]:
''' 형태소 분석 '''
nouns = []
verbs = []
adjs = []
josas = [] # 불용어 목록에 조사 사용하기 위함.
morphs = [] # 품사 태깅 없이 형태소만 추출. 추후 TF-IDF, N-gram등 텍스트 분석은 이 데이터로 진행함.
pos_list = [] # 감성분석할 때 필요.

In [4]:
for i in tqdm(range(len(corpus))):
    # 답변에서 형태소/품사 추출
    try:
        a = okt.pos(corpus[i], norm=True, stem=True) # 단어의 정규화와 어간 추출을 실행(True).
        m = okt.morphs(corpus[i], norm=True, stem=False)
        morphs.append(m) # 형태소 추가
        pos_list.append(a) # 형태소/품사 추가
        for x, y in a:
            # 품사가 명사면 명사 리스트에 단어 추가
            if y == 'Noun':
                nouns.append(x)
            # 품사가 동사면 동사 리스트에 단어 추가
            elif y == 'Verb':
                verbs.append(x)
            # 품사가 형용사면 형용사 리스트에 단어 추가
            elif y == 'Adjective':
                adjs.append(x)
		    # 품사가 조사면 조사 리스트에 단어 추가.
            elif y == 'Josa':
                josas.append(x)
    except:
        pass

100%|██████████| 1084190/1084190 [22:39:22<00:00, 13.29it/s]   


In [5]:
# 명사, 동사, 형용사 리스트에서 10개 단어씩 출력해서 확인
print(nouns[0:10])
print(verbs[0:10])
print(adjs[0:10])
print(josas[0:10])

['과천', '서울대', '공원', '딜쿠샤', '철도', '노조', '파업', '수요일', '오늘', '과천']
['살다', '만나다', '걷다', '하다', '두다', '들다', '오다', '지다', '먹다', '가다']
['있다', '있다', '있다', '기쁘다', '특이하다', '풍부하다', '있다', '유명하다', '있다', '있다']
['하고', '에', '가', '은', '을', '를', '는', '이', '하고', '를']


In [6]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(nouns))) # 이 값을 밑에 most_common 안에 넣으면 됨.
noun_cnt = Counter(nouns).most_common()

명사 = []
명사빈도 = []
for a, b in noun_cnt:
    명사.append(a)
    명사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
noun_df = pd.DataFrame({'명사':명사, '명사빈도':명사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
noun_df.to_csv("../dataset/2022/본문_형태소분석/11월/나무문화_11월_형태소분석(명사).csv", index=False, encoding='UTF-8')
noun_df

165622


,명사,명사빈도
0,것,234674
1,수,141692
2,그,119369
3,이,112520
4,곳,88603
...,...,...
165617,움뫔마,1
165618,퍼는데,1
165619,우두둑,1
165620,뀄던,1


In [7]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(verbs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
verb_cnt = Counter(verbs).most_common()

동사 = []
동사빈도 = []
for a, b in verb_cnt:
    동사.append(a)
    동사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
verb_df = pd.DataFrame({'동사':동사, '동사빈도':동사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
verb_df.to_csv("../dataset/2022/본문_형태소분석/11월/나무문화_11월_형태소분석(동사).csv", index=False, encoding='UTF-8')
verb_df

1893


,동사,동사빈도
0,하다,1000489
1,되다,201010
2,보다,122387
3,않다,92766
4,되어다,88330
...,...,...
1888,허청대다,1
1889,터벅거리다,1
1890,휘정거리다,1
1891,뉘엿대다,1


In [8]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(adjs))) # 이 값을 밑에 most_common 안에 넣으면 됨.
adj_cnt = Counter(adjs).most_common()

형용사 = []
형용사빈도 = []
for a, b in adj_cnt:
    형용사.append(a)
    형용사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
adj_df = pd.DataFrame({'형용사':형용사, '형용사빈도':형용사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
adj_df.to_csv("../dataset/2022/본문_형태소분석/11월/나무문화_11월_형태소분석(형용사).csv", index=False, encoding='UTF-8')
adj_df

1872


,형용사,형용사빈도
0,있다,471089
1,없다,86817
2,좋다,83399
3,같다,75719
4,이다,66863
...,...,...
1867,멀쑥하다,1
1868,섬약하다,1
1869,미끄덩하다,1
1870,우부룩하다,1


In [9]:
''' 명사, 동사, 형용사, 조사 똑같이 아래 코드 반복 '''
# 명사 리스트에서 명사 빈도 리스트 생성
print(len(Counter(josas))) # 이 값을 밑에 most_common 안에 넣으면 됨.
josa_cnt = Counter(josas).most_common()

조사 = []
조사빈도 = []
for a, b in josa_cnt:
    조사.append(a)
    조사빈도.append(b)

# 단어와 빈도를 가지고 판다스 데이터프레임(엑셀 표와 비슷) 생성
josa_df = pd.DataFrame({'조사':조사, '조사빈도':조사빈도})
# 명사 데이터프레임을 파일로 저장. .xlsx, .csv, .txt 등 가능.
josa_df.to_csv("../dataset/2022/본문_형태소분석/11월/나무문화_11월_형태소분석(조사).csv", index=False, encoding='UTF-8')
josa_df

399


,조사,조사빈도
0,을,800958
1,의,633497
2,이,624052
3,에,594318
4,를,391526
...,...,...
394,이라고만은,1
395,에까지만,1
396,으로랑,1
397,과만은,1


In [10]:
df_concat = pd.concat([noun_df,verb_df,adj_df],axis=1)
df_concat.to_csv("../dataset/2022/본문_형태소분석/나무문화_11월_단어종합.txt", index=False, encoding='UTF-8')

In [11]:
df["morphs"] = morphs
df["pos_tag"] = pos_list
df

,split_str,org_idx,morphs,pos_tag
0,과천서울대공원 딜쿠샤 철도노조파업 2022 11 30 수요일 오늘 과천 사는 친구 ...,0,"[과천, 서울대, 공원, 딜쿠샤, 철도, 노조, 파업, 2022, 11, 30, 수...","[(과천, Noun), (서울대, Noun), (공원, Noun), (딜쿠샤, No..."
1,서울은 지하철 타기가 전쟁통을 방불케 한다,0,"[서울, 은, 지하철, 타기가, 전쟁, 통, 을, 방불, 케, 한다]","[(서울, Noun), (은, Josa), (지하철, Noun), (타다, Verb..."
2,그래도 질서 있게 줄 섰다가 다음 열차를 타라는 안내 방송이 나오면 타기를 중단하고...,0,"[그래도, 질서, 있게, 줄, 섰다가, 다음, 열차, 를, 타라, 는, 안내, 방송...","[(그래도, Adverb), (질서, Noun), (있다, Adjective), (..."
3,서울대공원 안의 호수 메타세쿼이아 길 딜쿠샤는 기쁜 마음 이라는 뜻의 페르시아어로 ...,0,"[서울대, 공원, 안, 의, 호수, 메타세쿼이아, 길, 딜쿠샤, 는, 기쁜, 마음,...","[(서울대, Noun), (공원, Noun), (안, Noun), (의, Josa)..."
4,앨버트 테일러는 3 1운동을 뉴욕타임즈에 보도함으로써 우리 나라의 항일운동을 세계에...,0,"[앨버트, 테일러, 는, 3, 1, 운동, 을, 뉴욕타임즈, 에, 보도, 함, 으로...","[(앨버트, Noun), (테일러, Noun), (는, Josa), (3, Numb..."
...,...,...,...,...
1084185,갑신정변이 실패하고 홍영식이 비참하게 죽은 뒤 민영익의 도움을 받은 알렌은 흉가가 ...,22958,"[갑신정변, 이, 실패하고, 홍영식, 이, 비참하게, 죽은, 뒤, 민영익, 의, 도...","[(갑신정변, Noun), (이, Josa), (실패하다, Adjective), (..."
1084186,제중원의 뜰에는 나무 한그루가 서 있었다,22958,"[제중원, 의, 뜰, 에는, 나무, 한그루, 가, 서, 있었다]","[(제중원, Noun), (의, Josa), (뜰, Noun), (에는, Josa)..."
1084187,박지원의 집앞에 있었던 나무 홍영식의 집앞에 있었던 나무 그날 길 잃은 아이들처럼 ...,22958,"[박지원, 의, 집앞, 에, 있었던, 나무, 홍영식, 의, 집앞, 에, 있었던, 나...","[(박지원, Noun), (의, Josa), (집앞, Noun), (에, Josa)..."
1084188,그녀와 내가 헤어진 지금 이 모든 일이 과연 우연일 뿐이라고 생각하는가 21쪽 쉽게...,22958,"[그녀, 와, 내, 가, 헤어진, 지금, 이, 모든, 일이, 과연, 우연, 일, 뿐...","[(그녀, Noun), (와, Josa), (내, Noun), (가, Josa), ..."


In [12]:
df.to_csv("../dataset/2022/본문_형태소분석/나무문화_본문_11월_형태소분석.txt", index=False, encoding='UTF-8')